# Zusammenfassung der Zwischenergebnisse

## Festgelegte Features

Die Auswahl weiterer Features für das Vorhersagemodell basiert auf ihrer logischen Relevanz und der Analyse ihrer Korrelation mit den Verkaufszahlen.

Der Wochentag (DayOfWeek) hat oft einen erheblichen Einfluss auf die Verkäufe, da die Kaufgewohnheiten der Kunden je nach Wochentag variieren können. Beispielsweise gibt es mehr Einkäufe an Wochenenden. Daher wird der Wochentag in Liste der verwendbaren Merkmale aufgenommen.

Das Merkmal Customers (Kundenanzahl) hingegen ist weniger geeignet. Diese Entscheidung basiert auf der Abwägung der Auswirkungen auf die Modellgenauigkeit und die Komplexität des Vorhersageprozesses. Die Einbeziehung dieses Features würde bedeuten, dass auch die Anzahl der Kunden für zukünftige Zeitpunkte vorhersagt werden müsste. Dies fügt eine zusätzliche Schicht von Komplexität hinzu, da nun zwei miteinander verknüpfte Vorhersagen getroffen werden müssten: die Anzahl der Kunden und darauf basierend die Verkaufszahlen. Die Vorhersage der Kundenanzahl würde zusätzliche Modelle oder Algorithmen erfordern, die ebenfalls eine gewisse Fehlerquote aufweisen. Diese Fehler würden sich in der Verkaufsprognose widerspiegeln und könnten die Genauigkeit des Verkaufsprognosemodells verringern. Durch den Ausschluss des Customers-Features wird eine potenzielle Fehlerquelle reduziert und der Fokus liegt auf direkt beobachtbaren und prognostizierbaren Merkmalen. Der bewusste Ausschluss dieses Features hilft dabei, die Vorhersagegenauigkeit zu maximieren und die Komplexität des Modells zu minimieren, was zu einem effizienteren und zuverlässigeren Vorhersagemodell führt.

Ob ein Geschäft geöffnet oder geschlossen ist (Open), hat einen direkten Einfluss auf die Verkäufe. Geschäfte, die geschlossen sind, haben keine Verkäufe. Die Korrelationsmatrix zeigt erwartungsgemäß eine hohe Korrelation mit Sales, da Verkäufe nur an geöffneten Tagen stattfinden können.

Promotionen (Promo) beeinflussen die Verkaufszahlen erheblich, indem sie mehr Kunden anziehen und den Umsatz steigern. Die Korrelationsmatrix weist darauf hin, dass Promotionen zu höheren Verkaufszahlen führen, da eine positive Korrelation mit Sales besteht.

Feiertage (StateHoliday) beeinflussen das Einkaufsverhalten der Kunden, da an staatlichen Feiertagen Geschäfte geschlossen sein oder weniger Kunden haben könnten. Die Korrelation variiert je nach Art des Feiertags, zeigt aber signifikante Zusammenhänge.

Unterschiedliche Store-Typen (StoreType) können unterschiedliche Verkaufsmuster haben. Größere Geschäfte oder spezielle Geschäftsmodelle könnten höhere Verkäufe generieren. Die Korrelationsmatrix zeigt, dass die Korrelation je nach Typ variiert, aber signifikante Korrelationen mit anderen wichtigen Features wie Promo bestehen.

Die Wahl der zusätzlichen Features basiert auf ihrer logischen Relevanz und bisherigen Korrelationsanalysen. Features wie DayOfWeek, Open, Promo, StateHoliday und StoreType sind stark mit den Verkaufszahlen verbunden und bieten wertvolle Informationen zur Verbesserung der Modellgenauigkeit. Eine detaillierte Untersuchung der Korrelationen dieser Features mit Sales bestätigt ihre Bedeutung und rechtfertigt ihre Einbeziehung in das Vorhersagemodell.

## Änderungen

### Ausschluss von Features

Um die relevanten Features für das Modell auszuwählen, wurden verschiedene Merkmale getestet, um deren Einfluss auf die Vorhersagegenauigkeit zu bewerten. Dabei zeigte sich, dass die **Lag-Features**, die Genauigkeit des Modells erheblich verschlechterten. Aufgrund dieses negativen Effekts wurden diese Features in den weiteren Modellentwicklungen weggelassen.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit

# RMSPE Funktion definieren
def rmspe(y_true, y_pred):
    y_pred = np.array(y_pred)  # Umwandlung der Vorhersagen in ein Numpy-Array
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# RMSPE als Scorer definieren
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Laden des Datensatzes
data_cleaned = "../data/cleaned_train_alt.csv"
data = pd.read_csv(data_cleaned, delimiter=",", encoding="latin", header=0, thousands=",", decimal='.', low_memory=False)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'lag_1', 'lag_7']
already_encoded_features = ['Open', 'Promo', 'promo2']
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday', 'Assortment']

# Preprocessing Pipeline für numerische und kategorische Features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ])

# Modell für RandomForest erstellen
model = RandomForestRegressor(n_estimators=1, random_state=42)

# Pipeline erstellen
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

#Funktion zur Vorhersage der nächsten x Tage (wie im Originalcode)
def predict_multiple_days_with_updates(X_last, test_data, num_days):
    predictions = []
    X_next = X_last.copy()
    X_next = pd.DataFrame([X_next])  # In DataFrame umwandeln
    last_lags = list(X_last[['lag_1']].values.flatten())  # Initialisiere letzte Lags

    for day in range(num_days):
        # Preprocess die Daten vor der Vorhersage
        X_next_preprocessed = pipeline.named_steps['preprocessor'].transform(X_next)
        
        # Vorhersage des nächsten Tages
        y_next_pred = pipeline.named_steps['model'].predict(X_next_preprocessed)[0]
        predictions.append(y_next_pred)

        # Aktualisiere die Lag-Features basierend auf der Vorhersage
        last_lags.append(y_next_pred)
        last_lags.pop(0)  # Aktualisiere Lag-Werte

        X_next.loc[:, 'lag_1'] = y_next_pred
        X_next.loc[:, 'lag_7'] = last_lags[0]

        # Aktualisiere andere Features (Tag, Monat, Woche)
        X_next.loc[:, 'day'] += 1
        if X_next.loc[:, 'day'].values[0] > 31:
            X_next.loc[:, 'day'] = 1
            X_next.loc[:, 'month'] += 1
        if X_next.loc[:, 'month'].values[0] > 12:
            X_next.loc[:, 'month'] = 1
            X_next.loc[:, 'year'] += 1

        X_next.loc[:, 'week_of_year'] = (X_next['week_of_year'] % 52) + 1
        X_next.loc[:, 'DayOfWeek'] = (X_next['DayOfWeek'] % 7) + 1

        # Promo- und andere Features aus Testdaten übernehmen
        if day < len(test_data):
            X_next.loc[:, 'Promo'] = test_data.iloc[day]['Promo']
            X_next.loc[:, 'Open'] = test_data.iloc[day]['Open']
            X_next.loc[:, 'StateHoliday'] = test_data.iloc[day]['StateHoliday']

    return predictions

# TimeSeriesSplit für Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Überprüfung mit Cross-Validation
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

cv_rmspe_scores = []
for train_idx, test_idx in tscv.split(X_train):
    X_train_cv, X_test_cv = X_train.iloc[train_idx], X_train.iloc[test_idx]
    y_train_cv, y_test_cv = y_train.iloc[train_idx], y_train.iloc[test_idx]

    # Pipeline auf den Cross-Validation-Trainingsdaten fitten
    pipeline.fit(X_train_cv, y_train_cv)

    # Lag-Feature-Vorhersage für die CV-Testdaten
    X_last_cv = X_test_cv.iloc[0]  # Starte mit dem ersten Testtag
    num_days_cv = len(X_test_cv)    # Vorhersage für alle Testtage
    future_predictions_cv = predict_multiple_days_with_updates(X_last_cv, X_test_cv, num_days_cv)

    # Berechne RMSPE für diesen Split
    rmspe_score = rmspe(y_test_cv.values, future_predictions_cv)
    cv_rmspe_scores.append(rmspe_score)

# Ausgabe der Ergebnisse
mean_cv_rmspe = np.mean(cv_rmspe_scores)
print(f"Durchschnittliche RMSPE (Cross-Validation): {mean_cv_rmspe:.4f}")

# Nimm die letzten Daten als Startpunkt für die Vorhersage
X_last = X_test.iloc[-60]

# Vorhersage für die nächsten 7 Tage unter Verwendung von Promo-Informationen aus dem Testdatensatz
num_days = 60
test_data_next_days = X_test.iloc[:num_days]  # Extrahiere die Testdaten für die nächsten Tage

# Vorhersagen für die nächsten Tage machen
future_predictions = predict_multiple_days_with_updates(X_last, test_data_next_days, num_days)

# Ergebnisse anzeigen
print(f"Vorhersagen für die nächsten {num_days} Tage:", future_predictions)

# Tatsächliche Verkaufszahlen für die nächsten 7 Tage aus dem Testdatensatz extrahieren
actual_sales_next_days = y_test.iloc[:num_days].values

# Berechnung der RMSPE für die Vorhersagen
prediction_rmspe = rmspe(actual_sales_next_days, future_predictions)

# Ergebnis anzeigen
print(f"RMSPE der Vorhersagen für die nächsten {num_days} Tage: {prediction_rmspe:.4f}")

Durchschnittliche RMSPE (Cross-Validation): 1.7539
Vorhersagen für die nächsten 60 Tage: [6868.0, 6235.0, 5694.0, 0.0, 0.0, 9615.0, 8180.0, 6480.0, 6328.0, 5470.0, 0.0, 4090.0, 3776.0, 3756.0, 3973.0, 3476.0, 3169.0, 0.0, 9615.0, 10744.0, 10041.0, 8735.0, 9165.0, 8373.0, 0.0, 15301.0, 25107.0, 16837.0, 18127.0, 16058.0, 16077.0, 0.0, 8324.0, 8328.0, 8328.0, 6355.0, 6594.0, 6588.0, 0.0, 4090.0, 3646.0, 3309.0, 2723.0, 3333.0, 2685.0, 0.0, 9615.0, 10311.0, 10216.0, 8735.0, 9165.0, 8373.0, 0.0, 4090.0, 3646.0, 3309.0, 2930.0, 3848.0, 4086.0, 0.0]
RMSPE der Vorhersagen für die nächsten 60 Tage: 0.9235


### Neue Features 

Durch zuvor erwähnten Tests der unterschiedlichen Feature-Kombinationen wurde der Einfluss der einzelnen Variablen auf die Modellleistung ermittelt. Basierend auf diesen Erkenntnissen wurden die finalen Features für das Modell festgelegt, da sie die bestmögliche Leistung ohne weitere Optimierungen erreichten.

Die Zeitmerkmale Jahr, Monat, Tag und Woche des Jahres helfen dabei, saisonale Trends und zeitabhängige Muster zu erfassen. Verkäufe variieren häufig in bestimmten Monaten oder Jahreszeiten, was die Vorhersagegenauigkeit deutlich erhöht. Die Verwendung von Fourier-Transformationen, speziell die Fourier-Sin- und Fourier-Cos-Funktionen für 365 Tage, ermöglicht es, saisonale Effekte auf kontinuierliche Weise zu modellieren. Diese Features sind besonders nützlich, um komplexe saisonale Muster zu identifizieren und verbessern die Robustheit der Vorhersagen.
Die Merkmale `days_since_last_holiday` und `days_until_next_holiday` `wurden gewählt, da Feiertage oft einen signifikanten Einfluss auf die Verkaufszahlen haben. Diese zeitlichen Angaben helfen dem Modell, den Einfluss von Feiertagen auf den Verkauf in den relevanten Zeiträumen zu erfassen.

`Promo` und `Promo2` geben an, ob während bestimmter Zeiträume Werbeaktionen durchgeführt werden. Promotionen haben häufig einen erheblichen Einfluss auf die Verkaufszahlen, und die Berücksichtigung dieser Merkmale ermöglicht es dem Modell, den Effekt von Rabatten auf die Verkaufsprognosen zu lernen.

Verkaufszahlen variieren oft an verschiedenen Wochentagen, und die Codierung des Features `day_of_the_week` erlaubt es dem Modell, wöchentliche Muster zu erkennen. Das `StoreType`-Feature ist relevant, da die Art des Geschäfts einen Einfluss auf die Verkaufszahlen haben kann. Unterschiedliche Geschäftsmodelle ziehen unterschiedliche Kunden an und verfolgen variierende Verkaufsstrategien.

Ein weiteres wichtiges Feature ist `StateHoliday`, das angibt, ob ein Feiertag vorliegt. Feiertage haben oft einen starken Einfluss auf den Einzelhandel, und das Modell kann lernen, diese Effekte zu berücksichtigen. Das `Assortment`-Feature, das die Produktvielfalt im Geschäft beschreibt, ist ebenfalls entscheidend, da unterschiedliche Sortimentstypen verschiedene Kunden anziehen und die Verkaufszahlen beeinflussen können. Schließlich liefert das `Store`-Feature Informationen über spezifische Geschäfte, was hilft, lokale Verkaufsunterschiede zu erfassen. Jedes Geschäft kann unterschiedliche Verkaufsleistungen aufweisen, die durch Faktoren wie Lage, Größe und Zielgruppe bedingt sind.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import make_scorer

from sklearn.impute import SimpleImputer

# Zusätzliche Features erstellen
data['is_month_end'] = data['day'].isin([28, 29, 30, 31]).astype(int)
data['is_quarter_end'] = data['month'].isin([3, 6, 9, 12]).astype(int)
data['sin_day_of_week'] = np.sin(2 * np.pi * data['day_of_week'] / 7)
data['cos_day_of_week'] = np.cos(2 * np.pi * data['day_of_week'] / 7)

# Zielvariable und Features definieren
X = data.drop('Sales', axis=1)
y = data['Sales']

df = pd.DataFrame(data)
# Bestimme die Grenze für 80% Training und 20% Test
train_size = int(len(df) * 0.8)
# Aufteilen in Trainings- und Test-Datensätze
train = df.iloc[:train_size]
test = df.iloc[train_size:]

# Definiere die numerischen und kategorischen Features
numerical_features = ['year', 'month', 'day', 'week_of_year', 'is_month_end','is_quarter_end', 'sin_day_of_week', 'cos_day_of_week']

# Bereits encodierte Features
already_encoded_features = ['Open', 'Promo', 'promo2']

# Noch nicht encodierte kategorische Features
categorical_features_to_encode = ['Store', 'DayOfWeek', 'StoreType', 'StateHoliday','Assortment']

# Definiere einen Imputer für numerische Daten
numerical_imputer = SimpleImputer(strategy='mean')  # Oder 'median', je nach Bedarf

# Füge den Imputer zur Preprocessing-Pipeline hinzu
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[('imputer', numerical_imputer), ('scaler', StandardScaler())]), numerical_features+ already_encoded_features),
        ('enc', 'passthrough', already_encoded_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features_to_encode)
    ]
)

# RMSPE Funktion definieren
def rmspe(y_true, y_pred):
    mask = y_true != 0
    y_true_filtered = y_true[mask]
    y_pred_filtered = y_pred[mask]
    return np.sqrt(np.mean(((y_true_filtered - y_pred_filtered) / y_true_filtered) ** 2))

# Scorer für Cross-Validation
rmspe_scorer = make_scorer(rmspe, greater_is_better=False)

# Daten in Training (80%) und Test (20%) aufteilen
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# TimeSeriesSplit für Cross-Validation
tscv = TimeSeriesSplit(n_splits=5)

# Teste verschiedene Werte für n_estimators
n_estimators_range = [1, 5, 10, 20]
train_rmspe_scores = []
test_rmspe_scores = []

for n in n_estimators_range:
    # Erstelle die Pipeline mit dem aktuellen Wert für n_estimators
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', RandomForestRegressor(n_estimators=n, random_state=42))
    ])
    
    # Cross-Validation auf dem Trainingsdatensatz durchführen
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=tscv, scoring=rmspe_scorer)
    mean_cv_score = -np.mean(cv_scores)  # Da RMSPE negativ definiert ist
    
    # Trainiere das Modell auf dem gesamten Trainingsdatensatz
    pipeline.fit(X_train, y_train)
    
    # Vorhersagen auf dem Testdatensatz
    y_test_pred = pipeline.predict(X_test)
    
    # Berechne RMSPE für den Testdatensatz
    test_rmspe = rmspe(y_test, y_test_pred)
    
    # Speichere die Scores
    train_rmspe_scores.append(mean_cv_score)
    test_rmspe_scores.append(test_rmspe)
    
    # Ausgabe der Ergebnisse
    print(f"n_estimators: {n} - Train RMSPE: {mean_cv_score:.4f}, Test RMSPE: {test_rmspe:.4f}")

# Plotten der Ergebnisse
plt.figure(figsize=(10, 6))
plt.plot(n_estimators_range, train_rmspe_scores, label='Train RMSPE (Cross-Validation)', marker='o')
plt.plot(n_estimators_range, test_rmspe_scores, label='Test RMSPE', marker='o')
plt.xlabel('n_estimators')
plt.ylabel('RMSPE')
plt.title('RMSPE vs. n_estimators (TimeSeriesSplit)')
plt.legend()
plt.grid(True)
plt.show()


n_estimators: 1 - Train RMSPE: 0.4084, Test RMSPE: 0.3796
n_estimators: 5 - Train RMSPE: 0.3983, Test RMSPE: 0.3716
n_estimators: 10 - Train RMSPE: 0.3963, Test RMSPE: 0.3718


In [ ]:
# !!!!!!!!!neue features ausprobiere
# Definiere die numerischen und kategorischen Features
numerical_features = ['month', 'week_of_year', 'is_weekend', 'day_of_year', 'day_of_week']
already_encoded_features = ['Promo', 'promo2']
categorical_features_to_encode = ['StoreType', 'Store']

In [ ]:
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=tscv, scoring=rmspe_scorer, verbose=True)
cv_scores

array([-0.40544973, -0.39666004, -0.41508695, -0.40354785, -0.40712357])

In [ ]:
mean_rmspe = np.mean(cv_scores)
mean_rmspe

-0.40557362686514165

In [ ]:
y_pred = pipeline.predict(X_test)
test_rmspe = rmspe(y_test, y_pred)
test_rmspe

0.37880351499349507